## DEEP MLP

| Iteration | Dropout | L2 Reg. | BatchNorm | EarlyStopping | LR Scheduler      | Scaling                        | Physics Features | Notes                          |
| --------- | ------- | ------- | --------- | ------------- | ----------------- | ------------------------------ | ---------------- | ------------------------------ |
| **1**     | 0.2     | 1e-4    | Yes       | Yes           | ReduceLROnPlateau | StdScaler (X) + Ringing→MinMax | Yes              | Strong backbone, best Deep MLP |
| **2**     | —       | 1e-4    | Yes       | Yes           | ReduceLROnPlateau | StdScaler (X) + Ringing→MinMax | Yes              | No dropout, heavier reg.       |


In [1]:
# ==================== ITERATION 1: STRONGER DEEP MLP (Embedding + Physics Branch) ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

# ============== SETTINGS ==============
SEED = 42
UNSEEN_PART = 'C2M0040120D'
BASE_DIR = r"C:\Users\pc\Desktop\Neural_Network_Models\Deep MLP\iteration_1"

os.makedirs(f"{BASE_DIR}/r2_rmse_tables", exist_ok=True)
os.makedirs(f"{BASE_DIR}/train_val_loss_curves", exist_ok=True)
os.makedirs(f"{BASE_DIR}/predicted_vs_actual", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)

# ============== LOAD DATA ==============
df = pd.read_csv(r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs_25percent_balanced.csv")

# ============== TARGETS + DROPS ==============
TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID', 'MOSFET']  # keep Part_Number for embedding

# ============== SPLIT SEEN/UNSEEN DEVICES ==============
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df = df[df['Part_Number'] == UNSEEN_PART].copy()

# ============== DERIVED PHYSICS FEATURES ==============
def compute_physics_features(row):
    L_eq = row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum()
    C_eq = row.get("Coss", 1e-12)
    f_res = 1/(2*np.pi*np.sqrt(L_eq*C_eq))/1e6 if L_eq>0 and C_eq>0 else 0
    overshoot_est = row["VDS_max"] - row["Vbus"]
    undershoot_est = 0 - row["VGS_th_min"]
    dVdt_est = row["VDS_max"]/row["Tp1"] if row["Tp1"]!=0 else 0
    dIdt_est = row["ID_max_25C"]/row["Tp1"] if row["Tp1"]!=0 else 0
    return pd.Series([f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est])

for df_ in [train_df, test_df]:
    df_[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']] = df_.apply(compute_physics_features, axis=1)

# We exclude Part_Number from numeric features, since it's handled by embedding
DROP_COLUMNS = ['DeviceID', 'MOSFET', 'Part_Number']

physics_features = ['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS + DROP_COLUMNS] + physics_features


# ============== SCALE INPUTS ==============
input_scaler = StandardScaler()
input_scaler.fit(pd.concat([train_df[INPUT_COLUMNS], test_df[INPUT_COLUMNS]]))
X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS])
X_test_all = input_scaler.transform(test_df[INPUT_COLUMNS])

# Separate physics columns (scaled)
phys_idx = [INPUT_COLUMNS.index(c) for c in physics_features]
X_train_phys = X_train_all[:, phys_idx]
X_test_phys = X_test_all[:, phys_idx]
X_train_main = np.delete(X_train_all, phys_idx, axis=1)
X_test_main = np.delete(X_test_all, phys_idx, axis=1)

# ============== DEVICE EMBEDDING ==============
part_lookup = {p:i for i,p in enumerate(pd.concat([train_df['Part_Number'], test_df['Part_Number']]).unique())}
train_parts = train_df['Part_Number'].map(part_lookup).values
test_parts = test_df['Part_Number'].map(part_lookup).values
n_devices = len(part_lookup)

# ============== SCALE OUTPUTS ==============
output_scalers, y_train_scaled, y_test_scaled = {}, pd.DataFrame(), pd.DataFrame()
for col in TARGET_COLUMNS:
    scaler = MinMaxScaler() if col == 'ringing_frequency_MHz' else StandardScaler()
    y_train_scaled[col] = scaler.fit_transform(train_df[[col]]).flatten()
    y_test_scaled[col] = scaler.transform(test_df[[col]]).flatten()
    output_scalers[col] = scaler

# ============== SPLIT TRAIN/VAL (70/15/15) ==============
Xtr_main, Xval_main, Xtr_phys, Xval_phys, ytr, yval, ptr, pval = train_test_split(
    X_train_main, X_train_phys, y_train_scaled.values, train_parts,
    test_size=0.15, random_state=SEED
)

# ============== DEFINE ANN =================
def build_ann(input_dim_main, input_dim_phys, n_devices, output_dim, dropout=0.2, l2_reg=1e-4):
    # Main numeric branch
    inp_main = layers.Input(shape=(input_dim_main,), name="main_inputs")
    x_main = layers.Dense(256, kernel_regularizer=regularizers.l2(l2_reg))(inp_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main); x_main = layers.Dropout(0.3)(x_main)
    x_main = layers.Dense(128, kernel_regularizer=regularizers.l2(l2_reg))(x_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main); x_main = layers.Dropout(0.2)(x_main)

    # Physics branch
    inp_phys = layers.Input(shape=(input_dim_phys,), name="physics_inputs")
    x_phys = layers.Dense(32, activation='relu')(inp_phys)
    x_phys = layers.Dense(16, activation='relu')(x_phys)

    # Device embedding branch
    inp_part = layers.Input(shape=(1,), name="device_input")
    emb = layers.Embedding(input_dim=n_devices, output_dim=8)(inp_part)
    emb = layers.Flatten()(emb)

    # Fusion
    x = layers.concatenate([x_main, x_phys, emb])
    x = layers.Dense(64, kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x); x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='relu')(x)

    out = layers.Dense(output_dim, activation='linear')(x)
    model = models.Model(inputs=[inp_main, inp_phys, inp_part], outputs=out)
    return model

model = build_ann(Xtr_main.shape[1], Xtr_phys.shape[1], n_devices, ytr.shape[1])
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=1e-3, first_decay_steps=50)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.Huber(), metrics=['mae'])

# ============== TRAINING ==============
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit([Xtr_main, Xtr_phys, ptr], ytr,
                    validation_data=([Xval_main, Xval_phys, pval], yval),
                    epochs=300, batch_size=128, callbacks=[early_stop], verbose=1)

model.save(f"{BASE_DIR}/models/iteration1_Deep_mlp.h5")

# ============== SAVE LOSS CURVE ==============
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("Huber Loss")
plt.title("Train vs Validation Loss")
plt.legend(); plt.savefig(f"{BASE_DIR}/train_val_loss_curves/loss.png"); plt.close()

# ============== EVALUATION FUNCTION (POSITIVE FILTER) ==============
def evaluate_and_save(X_main, X_phys, parts, y_scaled, name, positive_only=False):
    y_pred_scaled = model.predict([X_main, X_phys, parts])
    results = []
    for i, col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:, i].reshape(-1, 1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled[:, i].reshape(-1, 1)).flatten()
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        results.append((col, r2, rmse))

    df_results = pd.DataFrame(results, columns=["Target", "R2", "RMSE"]).sort_values("R2", ascending=False)

    # Apply filter if requested
    if positive_only:
        df_results = df_results[df_results["R2"] > 0]

    print(f"\nEvaluation on {name}:\n", df_results)
    df_results.to_csv(f"{BASE_DIR}/r2_rmse_tables/{name}.csv", index=False)
    return df_results

# ============== RUN EVALUATIONS ==============
evaluate_and_save(Xtr_main, Xtr_phys, ptr, ytr, "train")
evaluate_and_save(Xval_main, Xval_phys, pval, yval, "val")
evaluate_and_save(X_train_main, X_train_phys, train_parts, y_train_scaled.values, "test")
evaluate_and_save(X_test_main, X_test_phys, test_parts, y_test_scaled.values, "unseen", positive_only=True)

# ============== SCATTER PLOTS FOR TEST SPLIT ==============
def plot_predicted_vs_actual(X_main, X_phys, parts, y_scaled, name="test"):
    y_pred_scaled = model.predict([X_main, X_phys, parts])
    n_targets = len(TARGET_COLUMNS)
    n_cols = 3
    n_rows = int(np.ceil(n_targets / n_cols))

    plt.figure(figsize=(15, 5 * n_rows))
    for i, col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:, i].reshape(-1, 1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled[:, i].reshape(-1, 1)).flatten()

        plt.subplot(n_rows, n_cols, i + 1)
        plt.scatter(y_true, y_pred, alpha=0.5, s=10, color="navy")
        min_val = min(y_true.min(), y_pred.min())
        max_val = max(y_true.max(), y_pred.max())
        plt.plot([min_val, max_val], [min_val, max_val], "r--")  # diagonal reference
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.title(col)

    plt.tight_layout()
    plt.savefig(f"{BASE_DIR}/predicted_vs_actual/{name}_scatter.png")
    plt.close()

# Run scatter plot for test split
plot_predicted_vs_actual(X_train_main, X_train_phys, train_parts, y_train_scaled.values, name="test")



Epoch 1/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.1871 - mae: 0.4159 - val_loss: 0.0475 - val_mae: 0.1451
Epoch 2/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0659 - mae: 0.2153 - val_loss: 0.0360 - val_mae: 0.1349
Epoch 3/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0488 - mae: 0.1881 - val_loss: 0.0349 - val_mae: 0.1456
Epoch 4/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0419 - mae: 0.1804 - val_loss: 0.0281 - val_mae: 0.1352
Epoch 5/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0351 - mae: 0.1673 - val_loss: 0.0218 - val_mae: 0.1098
Epoch 6/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0333 - mae: 0.1632 - val_loss: 0.0241 - val_mae: 0.1277
Epoch 7/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0339 - mae: 0.1669 - val_loss: 0.0233 - val_mae: 0.1266
Epoch 8/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0315 - mae: 0.1597 - val_loss: 0.0208 - val_mae: 0.1110
Epoch 9/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/

2294/2294 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step

Evaluation on train:
                       Target        R2          RMSE
3   voltage_fall_time_pulse2  0.998285  2.843000e-10
2   voltage_fall_time_pulse1  0.998166  2.943497e-10
7   current_fall_time_pulse2  0.997200  6.224576e-10
6   current_fall_time_pulse1  0.997166  6.281576e-10
12     ringing_frequency_MHz  0.988758  3.501306e+00
0   voltage_rise_time_pulse1  0.988384  4.366614e-10
11        undershoot_pulse_2  0.984018  1.733055e+00
10        undershoot_pulse_1  0.983182  1.776807e+00
8          overshoot_pulse_1  0.971055  2.080753e+00
4   current_rise_time_pulse1  0.934541  1.257213e-08
9          overshoot_pulse_2  0.932414  6.574963e+00
5   current_rise_time_pulse2  0.915204  7.313842e-09
1   voltage_rise_time_pulse2  0.897157  1.291412e-09
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Evaluation on val:
                       Target        R2          RMSE
3   voltage_fall_time_pulse2  0.998358  2.768047e-10
2   voltage_fall_time_pu

In [3]:
# ==================== ITERATION 2: PHYSICS + EMBEDDING + WEIGHTED LOSS ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

# ============== SETTINGS ==============
SEED = 42
UNSEEN_PART = 'C2M0040120D'
BASE_DIR = r"C:\Users\pc\Desktop\Neural_Network_Models\Deep MLP\iteration_2"
os.makedirs(f"{BASE_DIR}/r2_rmse_tables", exist_ok=True)
os.makedirs(f"{BASE_DIR}/train_val_loss_curves", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)

# ============== LOAD DATA ==============
df = pd.read_csv(r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs_25percent_balanced.csv")

# ============== TARGETS + DROPS ==============
TARGET_COLUMNS = [
    'voltage_rise_time_pulse1','voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1','voltage_fall_time_pulse2',
    'current_rise_time_pulse1','current_rise_time_pulse2',
    'current_fall_time_pulse1','current_fall_time_pulse2',
    'overshoot_pulse_1','overshoot_pulse_2',
    'undershoot_pulse_1','undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID','MOSFET']

# ============== SPLIT SEEN/UNSEEN DEVICES ==============
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df  = df[df['Part_Number']==UNSEEN_PART].copy()

# ============== DERIVED PHYSICS FEATURES ==============
def compute_physics_features(row):
    L_eq = row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum()
    C_eq = row.get("Coss", 1e-12)
    f_res = 1/(2*np.pi*np.sqrt(L_eq*C_eq))/1e6 if L_eq>0 and C_eq>0 else 0
    overshoot_est = row["VDS_max"] - row["Vbus"]
    undershoot_est = 0 - row["VGS_th_min"]
    dVdt_est = row["VDS_max"]/row["Tp1"] if row["Tp1"]!=0 else 0
    dIdt_est = row["ID_max_25C"]/row["Tp1"] if row["Tp1"]!=0 else 0
    return pd.Series([f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est])

for df_ in [train_df,test_df]:
    df_[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']] = df_.apply(compute_physics_features,axis=1)

DROP_COLUMNS = ['DeviceID','MOSFET','Part_Number']
physics_features = ['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS+DROP_COLUMNS] + physics_features

# ============== SCALE INPUTS ==============
input_scaler = StandardScaler()
input_scaler.fit(pd.concat([train_df[INPUT_COLUMNS], test_df[INPUT_COLUMNS]]))
X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS])
X_test_all  = input_scaler.transform(test_df[INPUT_COLUMNS])

phys_idx = [INPUT_COLUMNS.index(c) for c in physics_features]
X_train_phys = X_train_all[:,phys_idx]
X_test_phys  = X_test_all[:,phys_idx]
X_train_main = np.delete(X_train_all, phys_idx, axis=1)
X_test_main  = np.delete(X_test_all, phys_idx, axis=1)

# ============== DEVICE EMBEDDING ==============
part_lookup = {p:i for i,p in enumerate(pd.concat([train_df['Part_Number'],test_df['Part_Number']]).unique())}
train_parts = train_df['Part_Number'].map(part_lookup).values
test_parts  = test_df['Part_Number'].map(part_lookup).values
n_devices   = len(part_lookup)

# ============== SCALE OUTPUTS (hybrid) ==============
output_scalers,y_train_scaled,y_test_scaled = {},pd.DataFrame(),pd.DataFrame()
log_targets = ['voltage_rise_time_pulse1','voltage_rise_time_pulse2',
               'voltage_fall_time_pulse1','voltage_fall_time_pulse2',
               'current_rise_time_pulse1','current_rise_time_pulse2',
               'current_fall_time_pulse1','current_fall_time_pulse2',
               'ringing_frequency_MHz']
for col in TARGET_COLUMNS:
    if col == 'ringing_frequency_MHz':
        scaler = MinMaxScaler()
        y_train_scaled[col] = scaler.fit_transform(np.log1p(train_df[[col]])).flatten()
        y_test_scaled[col]  = scaler.transform(np.log1p(test_df[[col]])).flatten()
    elif col in log_targets:
        scaler = StandardScaler()
        y_train_scaled[col] = scaler.fit_transform(np.log1p(train_df[[col]])).flatten()
        y_test_scaled[col]  = scaler.transform(np.log1p(test_df[[col]])).flatten()
    else:
        scaler = StandardScaler()
        y_train_scaled[col] = scaler.fit_transform(train_df[[col]]).flatten()
        y_test_scaled[col]  = scaler.transform(test_df[[col]]).flatten()
    output_scalers[col] = scaler

# ============== TRAIN/VAL SPLIT ==============
Xtr_main,Xval_main,Xtr_phys,Xval_phys,ytr,yval,ptr,pval = train_test_split(
    X_train_main,X_train_phys,y_train_scaled.values,train_parts,
    test_size=0.15,random_state=SEED
)

# ============== WEIGHTED LOSS FUNCTION ==============
target_weights = {
    "overshoot_pulse_1": 2.0,
    "overshoot_pulse_2": 2.0,
    "undershoot_pulse_1": 2.0,
    "undershoot_pulse_2": 2.0,
    "ringing_frequency_MHz": 2.0
}
weights = np.array([target_weights.get(col,1.0) for col in TARGET_COLUMNS],dtype=np.float32)

def weighted_mse(y_true,y_pred):
    return tf.reduce_mean(weights * tf.square(y_true-y_pred))

# ============== DEFINE ANN =================
def build_ann(input_dim_main,input_dim_phys,n_devices,output_dim,dropout=0.2,l2_reg=1e-4):
    inp_main = layers.Input(shape=(input_dim_main,),name="main_inputs")
    x_main = layers.Dense(256,kernel_regularizer=regularizers.l2(l2_reg))(inp_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main); x_main = layers.Dropout(0.3)(x_main)
    x_main = layers.Dense(128,kernel_regularizer=regularizers.l2(l2_reg))(x_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main); x_main = layers.Dropout(0.2)(x_main)

    inp_phys = layers.Input(shape=(input_dim_phys,),name="physics_inputs")
    x_phys = layers.Dense(32,activation='relu')(inp_phys)
    x_phys = layers.Dense(16,activation='relu')(x_phys)

    inp_part = layers.Input(shape=(1,),name="device_input")
    emb = layers.Embedding(input_dim=n_devices,output_dim=8)(inp_part)
    emb = layers.Flatten()(emb)

    x = layers.concatenate([x_main,x_phys,emb])
    x = layers.Dense(64,kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x); x = layers.Dropout(0.1)(x)
    x = layers.Dense(32,activation='relu')(x)
    x = layers.Dense(16,activation='relu')(x)

    out = layers.Dense(output_dim,activation='linear')(x)
    return models.Model(inputs=[inp_main,inp_phys,inp_part],outputs=out)

model = build_ann(Xtr_main.shape[1],Xtr_phys.shape[1],n_devices,ytr.shape[1])
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=opt,loss=weighted_mse,metrics=['mae'])

# ============== TRAINING ==============
early_stop = callbacks.EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)
lr_sched = callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=5,min_lr=1e-5)

history = model.fit([Xtr_main,Xtr_phys,ptr],ytr,
                    validation_data=([Xval_main,Xval_phys,pval],yval),
                    epochs=300,batch_size=128,
                    callbacks=[early_stop,lr_sched],verbose=1)

model.save(f"{BASE_DIR}/models/iteration2_Deep_MLP.h5")


# ============== SAVE LOSS CURVE ==============
plt.figure()
plt.plot(history.history['loss'],label='Train Loss')
plt.plot(history.history['val_loss'],label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.title("Iteration 7 Loss Curve")
plt.legend(); plt.savefig(f"{BASE_DIR}/loss.png"); plt.close()

# ============== EVALUATION FUNCTION ==============
def evaluate_and_save(X_main,X_phys,parts,y_scaled,name,positive_only=False):
    y_pred_scaled = model.predict([X_main,X_phys,parts])
    results = []
    for i,col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:,i].reshape(-1,1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled[:,i].reshape(-1,1)).flatten()
        r2 = r2_score(y_true,y_pred); rmse = np.sqrt(mean_squared_error(y_true,y_pred))
        results.append((col,r2,rmse))
    df_results = pd.DataFrame(results,columns=["Target","R2","RMSE"]).sort_values("R2",ascending=False)
    if positive_only:
        df_results = df_results[df_results["R2"]>0]
    print(f"\nEvaluation on {name}:\n",df_results)
    df_results.to_csv(f"{BASE_DIR}{name}.csv",index=False)
    return df_results

# Run evaluations
evaluate_and_save(Xtr_main,Xtr_phys,ptr,ytr,"train")
evaluate_and_save(Xval_main,Xval_phys,pval,yval,"val")
evaluate_and_save(X_train_main,X_train_phys,train_parts,y_train_scaled.values,"test")
evaluate_and_save(X_test_main,X_test_phys,test_parts,y_test_scaled.values,"unseen",positive_only=True)


Epoch 1/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6759 - mae: 0.4995 - val_loss: 0.1122 - val_mae: 0.1647 - learning_rate: 0.0010
Epoch 2/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1451 - mae: 0.2050 - val_loss: 0.0853 - val_mae: 0.1341 - learning_rate: 0.0010
Epoch 3/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1231 - mae: 0.1864 - val_loss: 0.0740 - val_mae: 0.1261 - learning_rate: 0.0010
Epoch 4/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1086 - mae: 0.1757 - val_loss: 0.0738 - val_mae: 0.1356 - learning_rate: 0.0010
Epoch 5/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1006 - mae: 0.1718 - val_loss: 0.0751 - val_mae: 0.1447 - learning_rate: 0.0010
Epoch 6/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0970 - mae: 0.1678 - val_loss: 0.0596 - val_mae: 0.1162 - learning_rate: 0.0010
Epoch 7/300
574/574 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0903 - mae: 0.1618 - val_loss: 0.0609 - val_mae: 0.1230 - learning_rate: 0.0010

2294/2294 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step

Evaluation on train:
                       Target        R2          RMSE
2   voltage_fall_time_pulse1  0.998555  2.612718e-10
3   voltage_fall_time_pulse2  0.998549  2.614708e-10
6   current_fall_time_pulse1  0.995676  7.758541e-10
7   current_fall_time_pulse2  0.995422  7.958998e-10
0   voltage_rise_time_pulse1  0.991100  3.822191e-10
12     ringing_frequency_MHz  0.990367  3.836925e-02
11        undershoot_pulse_2  0.988283  1.483890e+00
10        undershoot_pulse_1  0.987324  1.542566e+00
8          overshoot_pulse_1  0.975528  1.913226e+00
9          overshoot_pulse_2  0.938914  6.250831e+00
4   current_rise_time_pulse1  0.934253  1.259904e-08
1   voltage_rise_time_pulse2  0.927841  1.081739e-09
5   current_rise_time_pulse2  0.922391  6.997019e-09
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Evaluation on val:
                       Target        R2          RMSE
2   voltage_fall_time_pulse1  0.998557  2.598308e-10
3   voltage_fall_time_pu

,Target,R2,RMSE
6,current_fall_time_pulse1,0.921716,3.141186e-09
7,current_fall_time_pulse2,0.915960,3.232239e-09
8,overshoot_pulse_1,0.782599,6.305406e+00
9,overshoot_pulse_2,0.748130,1.039741e+01
1,voltage_rise_time_pulse2,0.477300,3.372944e-09
11,undershoot_pulse_2,0.464133,7.132041e+00
10,undershoot_pulse_1,0.448310,7.285956e+00
5,current_rise_time_pulse2,0.404425,1.293814e-08
